<a href="https://colab.research.google.com/github/antoniobelotti/HVD/blob/main/3_augmented_vs_original_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup env

In [ ]:
%%capture

!python -m pip install --upgrade
!pip install --no-cache-dir transformers sentencepiece datasets
!pip install accelerate -U

import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas() # to use progress_apply
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import (
  AutoModelForSequenceClassification,
  AutoModel,
  AutoTokenizer,
  TrainingArguments,
  Trainer,
  EarlyStoppingCallback,
  IntervalStrategy,
  EvalPrediction,
  AdamW,
  get_linear_schedule_with_warmup
)
from datasets import (
  load_from_disk,
  DatasetDict,
  concatenate_datasets
)

import torch

from sklearn.model_selection import (
  StratifiedKFold,
  cross_validate,
  train_test_split
)
from sklearn.metrics import (
  make_scorer,
  accuracy_score,
  precision_score,
  recall_score,
  f1_score,
  confusion_matrix,
  classification_report
)

%matplotlib inline

In [ ]:
USE_DRIVE = True

In [ ]:
import pathlib

if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/gdrive')
  BASE_PATH = pathlib.Path("/content/gdrive/MyDrive/human_value_detection")
else:
  BASE_PATH = pathlib.Path("/content/human_value_detection")
  print("Manually upload the encoded datasets produced in notebook 1")

DATA_PATH = BASE_PATH / "data"

Mounted at /content/gdrive


In [ ]:
MODEL_NAME = "distilbert-base-uncased"
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LABEL_NAMES = ["Self-direction: thought", "Self-direction: action", "Stimulation", "Hedonism", "Achievement", "Power: dominance", "Power: resources", "Face", "Security: personal", "Security: societal", "Tradition", "Conformity: rules", "Conformity: interpersonal", "Humility", "Benevolence: caring", "Benevolence: dependability", "Universalism: concern", "Universalism: nature", "Universalism: tolerance", "Universalism: objectivity"]

# Classifiers

In [ ]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels):
  # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  metrics = {}

  for threshold in np.arange(0.1, 1, 0.05):
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    metrics[f'f1_macro_{threshold:.2f}'] = f1_score(labels, y_pred, average="macro")

  return {
    "f1_macro": max(metrics.values())
  }

def compute_metrics(p: EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
  result = multi_label_metrics(
    predictions=preds,
    labels=p.label_ids
  )
  return result

def model_init():
  return AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    problem_type="multi_label_classification",
    num_labels=20,
  )

def get_trainer(name, ds, seed=42):
  args = TrainingArguments(
    name,
    evaluation_strategy = IntervalStrategy.EPOCH,
    save_strategy = IntervalStrategy.EPOCH,
    save_total_limit = 1,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps = 2,
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True,
    push_to_hub=False,
    seed=seed
  )

  trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    tokenizer=TOKENIZER,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
  )

  return trainer

In [ ]:
n_retry = 5
reports = {}

for ds_name in ["original", "summarized", "masked", "backtranslated"]:
  dataset = load_from_disk(DATA_PATH / MODEL_NAME / ds_name)

  scores = np.zeros(n_retry)
  for i in range(n_retry):
    seed = int(time.time())

    trainer = get_trainer(ds_name, dataset, seed=seed)

    trainer.train()

    X_test = dataset["test"].remove_columns("labels")
    y_test = np.array(dataset["test"]["labels"], dtype="int")

    scores[i] = multi_label_metrics(
      trainer.predict(X_test).predictions,
      y_test
    )["f1_macro"]

  reports[ds_name] = scores.mean()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,No log,0.383802,0.295328
2,0.441100,0.349216,0.408594
2,0.313900,0.341492,0.411459
4,0.313900,0.355742,0.427921
4,0.238000,0.386320,0.423343
6,0.176000,0.415460,0.431947
6,0.176000,0.449380,0.423053
8,0.122900,0.469632,0.430176
8,0.092500,0.484553,0.428276


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,No log,0.380752,0.299585
2,0.438800,0.344567,0.412963
2,0.312500,0.340822,0.423341
4,0.312500,0.363554,0.422111
4,0.237900,0.381055,0.430369
6,0.178900,0.413175,0.422849
6,0.178900,0.437915,0.420249
8,0.124600,0.462586,0.423538


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,F1 Macro
0,No log,0.377587,0.285477
2,0.439400,0.343556,0.400854
2,0.308700,0.342232,0.414658
4,0.308700,0.356749,0.431125
4,0.230900,0.375304,0.436010
6,0.171100,0.405611,0.438677
6,0.171100,0.436230,0.432256
8,0.117400,0.452145,0.435094
8,0.090500,0.468253,0.432754


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,No log,0.372670,0.320631
2,0.441200,0.342319,0.399456
2,0.307100,0.336966,0.443717
4,0.307100,0.348283,0.436341
4,0.231900,0.369378,0.441190
6,0.172500,0.399953,0.438475


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,No log,0.379751,0.302931
2,0.439000,0.341307,0.400201
2,0.310400,0.338870,0.427006
4,0.310400,0.349386,0.434378
4,0.234700,0.375114,0.443857
6,0.174300,0.400703,0.438038
6,0.174300,0.431134,0.437231
8,0.122400,0.453810,0.439512


In [ ]:
reports

{'summarized': 0.40605220815372894,
 'masked': 0.3959099820166097,
 'backtranslated': 0.41934826711927614}

# All datasets

In [ ]:
import time

In [ ]:
n_retry = 5
reports = {}

dataset = load_from_disk(DATA_PATH / MODEL_NAME / "original")
for augmented_ds_name in ["summarized", "masked", "backtranslated"]:
  new_ds = load_from_disk(DATA_PATH / MODEL_NAME / augmented_ds_name)
  dataset["train"] = concatenate_datasets([dataset["train"], new_ds["train"]])

  scores = np.zeros(n_retry)
  for i in range(n_retry):
    seed = int(time.time())

    trainer = get_trainer("original-" + augmented_ds_name, dataset, seed=seed)

    trainer.train()

    X_test = dataset["test"].remove_columns("labels")
    y_test = np.array(dataset["test"]["labels"], dtype="int")

    scores[i] = multi_label_metrics(
      trainer.predict(X_test).predictions,
      y_test
    )["f1_macro"]

  reports["original-"+augmented_ds_name] = scores.mean()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,0.434800,0.346433,0.377523
2,0.307400,0.345617,0.437352
2,0.192300,0.399004,0.435172
4,0.133600,0.476115,0.425763
4,0.097500,0.542776,0.428585


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,0.441400,0.342338,0.401748
2,0.305900,0.357122,0.433282
2,0.192700,0.396788,0.439708
4,0.133600,0.475365,0.427451
4,0.096300,0.538812,0.437534
6,0.051000,0.595165,0.440375
6,0.038900,0.642109,0.430707
8,0.031100,0.669001,0.438215
8,0.023500,0.691702,0.433118


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,F1 Macro
0,0.320200,0.344961,0.430604
2,0.188000,0.422956,0.429310
2,0.087200,0.550503,0.429501
4,0.049200,0.675063,0.429190


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,0.315500,0.344691,0.424195
2,0.184800,0.419385,0.436658
2,0.086600,0.551387,0.437059
4,0.049900,0.670470,0.435948
4,0.028900,0.756898,0.432959
6,0.019200,0.820029,0.432194


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Macro
0,0.318900,0.340209,0.435177
2,0.185400,0.427473,0.432801
2,0.087900,0.557355,0.432919
4,0.049400,0.669812,0.437961
4,0.028600,0.737075,0.441044
6,0.019900,0.817395,0.437280
6,0.014900,0.860699,0.435561
8,0.011700,0.892253,0.434898


In [ ]:
reports

{'original-summarized': 0.42100764828944726,
 'original-masked': 0.4125954491651499,
 'original-backtranslated': 0.42191733925661473}